# SEGY 파일 처리 빠른 시작 (Google Colab)

이 노트북은 Google Colab에서 SEGY 파일을 빠르게 처리하기 위한 간단한 가이드입니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic_data_loading/blob/main/quickstart_colab.ipynb)

## 🚀 1단계: 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install segyio numpy matplotlib -q
print("✅ 라이브러리 설치 완료!")

In [ ]:
# GitHub에서 코드 가져오기
!git clone https://github.com/knocgp/seismic_data_loading.git
%cd seismic_data_loading
print("✅ 코드 다운로드 완료!")

In [ ]:
# 모듈 import
from header_loading import SEGYHeaderLoader
from data_loading import SEGYDataLoader
from data_divide import SEGYDataDivider
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✅ 모듈 import 완료!")

## 📁 2단계: SEGY 파일 준비

**세 가지 방법 중 하나를 선택하세요:**

### 방법 A: Google Drive 사용 (권장 ⭐)

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# Drive에 저장된 SEGY 파일 경로 설정
# ⚠️ 실제 파일 경로로 변경하세요!
segy_file = '/content/drive/MyDrive/SEGY_Data/your_file.segy'

# 파일 존재 확인
import os
if os.path.exists(segy_file):
    print(f"✅ 파일 찾음: {segy_file}")
else:
    print(f"❌ 파일 없음: {segy_file}")
    print("\n💡 Drive의 SEGY 파일 목록:")
    !find /content/drive/MyDrive -name "*.segy" -o -name "*.sgy" 2>/dev/null | head -10

### 방법 B: 파일 직접 업로드

In [ ]:
# 로컬에서 파일 업로드
from google.colab import files

print("SEGY 파일을 선택하세요...")
uploaded = files.upload()

segy_file = list(uploaded.keys())[0]
print(f"\n✅ 업로드 완료: {segy_file}")
print(f"파일 크기: {os.path.getsize(segy_file) / (1024*1024):.2f} MB")

### 방법 C: 샘플 데이터 다운로드

In [ ]:
# 공개된 샘플 SEGY 파일 다운로드
# 예시: 여기에 공개된 SEGY 파일 URL을 넣으세요
# !wget -O sample.segy "URL_TO_SEGY_FILE"
# segy_file = "sample.segy"

print("⚠️ 샘플 데이터 URL을 설정하고 실행하세요")

## 📊 3단계: 헤더 정보 확인

In [ ]:
# 헤더 정보 출력
with SEGYHeaderLoader(segy_file) as loader:
    loader.print_header_summary()
    info = loader.get_file_info()

# 주요 정보 변수로 저장
total_traces = info['total_traces']
samples_per_trace = info['samples_per_trace']
sample_interval_ms = info['sample_interval_ms']

print(f"\n📌 요약:")
print(f"   총 트레이스: {total_traces:,}")
print(f"   샘플/트레이스: {samples_per_trace:,}")
print(f"   샘플 간격: {sample_interval_ms} ms")

## 📈 4단계: 데이터 로드 및 시각화

In [ ]:
# 처음 100개 트레이스 로드
num_traces_to_load = min(100, total_traces)

with SEGYDataLoader(segy_file) as loader:
    data = loader.load_traces(0, num_traces_to_load)
    time_axis = loader.get_time_axis()
    
    print(f"로드된 데이터 형태: {data.shape}")
    print(f"시간 범위: {time_axis[0]:.2f} ~ {time_axis[-1]:.2f} ms")

# 시각화
plt.figure(figsize=(14, 8))
plt.imshow(data.T, aspect='auto', cmap='seismic',
           extent=[0, num_traces_to_load, time_axis[-1], time_axis[0]],
           vmin=-np.percentile(np.abs(data), 95),
           vmax=np.percentile(np.abs(data), 95))
plt.colorbar(label='Amplitude', shrink=0.8)
plt.xlabel('Trace Number', fontsize=12)
plt.ylabel('Time/Depth (ms)', fontsize=12)
plt.title(f'SEGY Data Preview (First {num_traces_to_load} Traces)', fontsize=14)
plt.tight_layout()
plt.show()

print("✅ 시각화 완료!")

## ✂️ 5단계: 데이터 분할 (옵션)

In [ ]:
# 분할 파라미터 설정
traces_per_chunk = 100  # 청크당 트레이스 수
depth_interval_ms = 500.0  # 깊이/시간 간격 (ms)

print(f"분할 설정:")
print(f"  청크당 트레이스: {traces_per_chunk}")
print(f"  깊이/시간 간격: {depth_interval_ms} ms")

# 데이터 분할 (정보만 확인)
with SEGYDataDivider(segy_file) as divider:
    chunks = divider.divide_by_grid(traces_per_chunk, depth_interval_ms)
    divider.print_division_info(chunks)

print(f"\n총 {len(chunks)}개 청크로 분할됩니다.")

### 청크 시각화

In [ ]:
# 첫 번째 청크 추출 및 시각화
if len(chunks) > 0:
    chunk = chunks[0]
    
    with SEGYDataDivider(segy_file) as divider:
        chunk_data = divider.extract_chunk(
            chunk['trace_range'],
            chunk['sample_range']
        )
    
    print(f"청크 #0 데이터 형태: {chunk_data.shape}")
    
    # 시각화
    t_start, t_end = chunk['trace_range']
    time_start, time_end = chunk['time_range_ms']
    
    plt.figure(figsize=(12, 6))
    plt.imshow(chunk_data.T, aspect='auto', cmap='seismic',
               extent=[t_start, t_end, time_end, time_start],
               vmin=-np.percentile(np.abs(chunk_data), 95),
               vmax=np.percentile(np.abs(chunk_data), 95))
    plt.colorbar(label='Amplitude')
    plt.xlabel('Trace Number')
    plt.ylabel('Time/Depth (ms)')
    plt.title(f'Chunk #0 Visualization')
    plt.tight_layout()
    plt.show()
    
    print("✅ 청크 시각화 완료!")

### 청크 저장 (Google Drive에 저장)

In [ ]:
# ⚠️ 청크를 저장하려면 실행하세요
SAVE_CHUNKS = False  # True로 변경하면 저장

if SAVE_CHUNKS:
    # Drive에 마운트되어 있는지 확인
    if not os.path.exists('/content/drive/MyDrive'):
        from google.colab import drive
        drive.mount('/content/drive')
    
    # 출력 디렉토리 설정
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f'/content/drive/MyDrive/SEGY_Chunks/{timestamp}'
    
    # 청크 저장
    with SEGYDataDivider(segy_file) as divider:
        divider.save_all_chunks(chunks, output_dir)
    
    print(f"\n✅ 청크 저장 완료: {output_dir}")
    print(f"총 {len(chunks)}개 파일 저장됨")
else:
    print("ℹ️ 청크 저장이 비활성화되어 있습니다.")
    print("저장하려면 SAVE_CHUNKS = True로 설정하세요.")

## 📊 6단계: 추가 분석

### 개별 트레이스 시각화

In [ ]:
# 여러 위치의 트레이스 비교
trace_indices = [0, total_traces // 4, total_traces // 2, total_traces * 3 // 4]
trace_indices = [t for t in trace_indices if t < total_traces]

fig, axes = plt.subplots(1, len(trace_indices), figsize=(16, 6), sharey=True)
if len(trace_indices) == 1:
    axes = [axes]

with SEGYDataLoader(segy_file) as loader:
    time_axis = loader.get_time_axis()
    
    for i, trace_idx in enumerate(trace_indices):
        trace_data = loader.load_trace(trace_idx)
        
        axes[i].plot(trace_data, time_axis, 'b-', linewidth=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, 
                             where=(trace_data > 0), color='red', alpha=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, 
                             where=(trace_data < 0), color='blue', alpha=0.5)
        axes[i].set_xlabel('Amplitude')
        axes[i].set_title(f'Trace #{trace_idx}')
        axes[i].grid(True, alpha=0.3)
        axes[i].invert_yaxis()

axes[0].set_ylabel('Time/Depth (ms)')
plt.tight_layout()
plt.show()

print("✅ 트레이스 시각화 완료!")

### 진폭 분포 분석

In [ ]:
# 샘플 데이터로 진폭 분석
sample_traces = min(1000, total_traces)

with SEGYDataLoader(segy_file) as loader:
    sample_data = loader.load_traces(0, sample_traces)

# 히스토그램과 박스 플롯
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램
axes[0].hist(sample_data.flatten(), bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Amplitude')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Amplitude Distribution')
axes[0].grid(True, alpha=0.3)

# 박스 플롯
axes[1].boxplot([sample_data.flatten()], vert=True)
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Amplitude Box Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 정보
print("\n진폭 통계:")
print(f"  최소값: {sample_data.min():.6e}")
print(f"  최대값: {sample_data.max():.6e}")
print(f"  평균: {sample_data.mean():.6e}")
print(f"  표준편차: {sample_data.std():.6e}")
print(f"  중앙값: {np.median(sample_data):.6e}")

print("\n✅ 진폭 분석 완료!")

## 🎉 완료!

### 다음 단계

1. **더 많은 기능 탐색**: `segy_processing_tutorial.ipynb` 참고
2. **문서 확인**: [README.md](https://github.com/knocgp/seismic_data_loading/blob/main/README.md)
3. **Colab 가이드**: [COLAB_GUIDE.md](https://github.com/knocgp/seismic_data_loading/blob/main/COLAB_GUIDE.md)

### 유용한 링크

- [GitHub 저장소](https://github.com/knocgp/seismic_data_loading)
- [이슈 등록](https://github.com/knocgp/seismic_data_loading/issues)
- [segyio 문서](https://segyio.readthedocs.io/)